# 1. Setup

In [3]:
# !pip3 install pandas
# !pip3 install numpy
# !pip3 install seaborn

  Using cached pandas-2.0.3-cp38-cp38-macosx_11_0_arm64.whl (10.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 9.4 MB/s eta 0:00:00a 0:00:01
  Using cached numpy-1.24.4-cp38-cp38-macosx_11_0_arm64.whl (13.8 MB)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 2.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 10.7 MB/s eta 0:00:00a 0:00:01
  Using cached kiwisolver-1.4.5-cp38-cp38-macosx_11_0_arm64.whl (66 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.1 MB/s eta 0:00:00
  Using cached contourpy-1.1.1-cp38-cp38-macosx_11_0_arm64.whl (232 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

# 2. Data Cleaning

## 2.2 Quick Look

In [6]:
# %ls
df = pd.read_csv('data.csv', encoding='ISO-8859-1') 

In [7]:
df.shape

(541909, 8)

In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

convert `InvoiceDate` into a `datetime` col

In [10]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

## 2.3 Anomaly Values

In [29]:
df[['Quantity', 'UnitPrice']].describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


rows with `UnitPrice` < 0 looks like book keeping adjustments, instead of actual customer purchasing records

In [31]:
df[df['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


`Quantity` < 0 also looks non-usable.

In [32]:
df[df['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


let's drop these

## NAs

25% (135,080 out of 541,909 rows) `customerID`'s and .2% (1,454 out of 541,909) are NAs!

In [11]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [15]:
df[df['CustomerID'].isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom


In [18]:
tmp = df[df['CustomerID'].isnull()]
tmp[['Quantity', 'UnitPrice']].describe()

,Quantity,UnitPrice
count,135080.000000,135080.000000
mean,1.995573,8.076577
std,66.696153,151.900816
min,-9600.000000,-11062.060000
25%,1.000000,1.630000
50%,1.000000,3.290000
75%,3.000000,5.450000
max,5568.000000,17836.460000


rows with negative `quantity` & `UnitPrice` values should be faulty values?

In [20]:
tmp[(tmp['Quantity'] < 0) | (tmp['UnitPrice'] < 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom


now  we see a lot of the negative quantities and missing customerIDs are result from faulty (e.g. missing/ smashed/ lost/ thrown away etc.) goods

In [22]:
tmp[(tmp['Quantity'] < 0) ]['Description'].unique()

array([nan, '?', 'ENAMEL MEASURING JUG CREAM',
       '4 TRADITIONAL SPINNING TOPS', 'S/6 WOODEN SKITTLES IN COTTON BAG',
       'ALPHABET STENCIL CRAFT', 'TRADITIONAL WOODEN SKIPPING ROPE',
       "POPPY'S PLAYHOUSE BATHROOM", 'MEASURING TAPE BABUSHKA RED',
       'ROUND SNACK BOXES SET OF 4 SKULLS', 'RED  HARMONICA IN BOX ',
       'ROTATING SILVER ANGELS T-LIGHT HLDR', "POPPY'S PLAYHOUSE KITCHEN",
       'CHRISTMAS LIGHTS 10 VINTAGE BAUBLES', '10 COLOUR SPACEBOY PEN',
       'PAPER CHAIN KIT LONDON', 'FELTCRAFT CHRISTMAS FAIRY',
       'CHRISTMAS CRAFT TREE TOP ANGEL',
       'ROUND SNACK BOXES SET OF 4 FRUITS ', 'MAGIC DRAWING SLATE PURDEY',
       'EIGHT PIECE DINOSAUR SET', 'FAIRY TALE COTTAGE NIGHTLIGHT',
       'check', 'damages', 'Bank Charges', 'SAMPLES', 'AMAZON FEE',
       'Manual', 'faulty', 'Dotcom sales', 'GUMBALL COAT RACK',
       'SET OF 3 CAKE TINS PANTRY DESIGN ', 'BINGO SET',
       'reverse 21/5/10 adjustment', 'OCEAN SCENT CANDLE IN JEWELLED BOX',
       'POSTAG

In [27]:
tmp[(tmp['UnitPrice'] < 0) ]['Description'].unique()

array(['Adjust bad debt'], dtype=object)

let's add a `loss` column to mark these entries (may drop later)

In [23]:
tmp['loss'] = False
tmp['loss'] = np.where(
    tmp['loss'] < 0,
    True,
    tmp['loss']
)

/var/folders/p6/smkl584n3310p662pxj_990c0000gn/T/ipykernel_32747/482809412.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['faulty'] = False
/var/folders/p6/smkl584n3310p662pxj_990c0000gn/T/ipykernel_32747/482809412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['faulty'] = np.where(


the largest quantity was 5568. That sounds a bit strange, when 75% percentile was 3

# Countries

# Customers

# Reference

- [Kaggle notebook: ecommerce sales forecast](https://www.kaggle.com/code/allunia/e-commerce-sales-forecast)
- [Customer Segmentation](https://www.kaggle.com/code/fabiendaniel/customer-segmentation)